In [7]:
# Nonlinear dynamics JC
# save wave pattern generation task example
# (note linear approximation part is not implemented yet!)
# AK, July-1-2022
# =========================================================================
%reset -f
import os, sys
from pprint import pprint as pp

# Define paths 
DIR_HERE = os.path.dirname(os.path.realpath("__file__"))
sys.path.append("../../PyTorch_library/")

# Load modules 
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchinfo import summary
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

# PyTorch related functions 
from PyTorch_data import D_sinewave_pattern
from PyTorch_utils import L_default_weights, Trainer 

# Check GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device used:", device)

%who

Device used: cpu
DIR_HERE	 D_sinewave_pattern	 F	 L_default_weights	 Trainer	 Variable	 datasets	 device	 nn	 
np	 optim	 os	 plt	 pp	 summary	 sys	 torch	 transforms	 



In [2]:
?D_sinewave_pattern

Init signature: D_sinewave_pattern(seed=612, max_sample=1000, n_time=50, n_freq=10)
Docstring:     
Sine wave pattern generation (continuous)
Using constant frequency value, generate sine waves
n_sample = max number of batch
n_time = how many time samples to make
n_freq = range of frequency of sine wave
(x) = generated dataset
(t) = continuous sine wave
File:           ~/Dropbox/w_LABWORKS/STUDYGROUPS/DeepLearningFromScratch/PyTorch_library/PyTorch_data.py
Type:           type
Subclasses:     


In [8]:
#------------------------------
# Simulate data
n_batch = 10
n_time = 40
n_freq = 10
data = D_sinewave_pattern(n_time = n_time, n_freq = n_freq, max_sample = n_batch)

# Data loader for training
train_dataloader = torch.utils.data.DataLoader(data,
                                            batch_size = n_batch,
                                            shuffle = True)
 
# Checking loader
for i, (inputs, labels) in (enumerate(train_dataloader)):print("done")
# plt.plot(labels[100])

done


In [9]:

#------------------------------
# Define hyperparameters
n_epochs = 2000         # how many times to repeat learning epochs
learning_rate = .001    # learning rate of the oprimizer

# Layer properties
n_input = 1             # constant frequency value at time n
n_output = 1            # sine wave value at time n
n_rnn_layers = 1        # number of rnn layers stacked
n_units_hidden = 200    # number of hidden units
rnn_nl = 'relu'         # nonlinear update function of rnn layer


#------------------------------
# Define neural network model (dense feedforward network)
class Net(nn.Module):
    def __init__(self, n_time, n_input, n_output, n_units_hidden, n_rnn_layers, rnn_nl):
        # Initialize module
        super(Net, self).__init__()
        self.n_time = n_time
        self.n_input = n_input
        self.n_output = n_output
        self.n_units_hidden = n_units_hidden
        self.n_rnn_layers = n_rnn_layers
        
        # Prepare layers
        self.rnn = nn.RNN(n_input, n_units_hidden, nonlinearity = rnn_nl, dropout = 0.0, batch_first = True)
        self.fc = nn.Linear(n_units_hidden, n_output)
        
        # Initialize weights (to use defalt comment out all)
        #self.apply(self._init_weights)
        #self.apply(L_default_weights) # from PyTorch_util
        
    def _init_weights(self, m):
        print("To modify initial weights modify this part")
        raise NotImplementedError()
        
    def _set_state(self):
        weight = next(self.parameters()).data
        return Variable(weight.new(self.n_rnn_layers, self.n_batch, self.n_units_hidden).zero_())
    
    # def forward
    
    def forward(self, x):  
        # Initialize hidden state
        self.n_batch = x.shape[0] # batch_first = True
        self.h_state = self._set_state()
        
        # Transform data: (Batch, Time, Input)
        x = x.view(self.n_batch, self.n_time, self.n_input)
        output = torch.zeros(self.n_batch, self.n_time, self.n_output).type_as(x.data)
        
        # Connect layers and pass on to activation functions
        #(rnn_out) = (batch, time, hidden_size)
        #(h_n) =  (num_layers, batch, hidden_size)
        h_all, h_n = self.rnn(x, self.h_state) # loop over n_time inside of rnn module
        
        # Hidden state at each time point is passed on to same output layer
        # https://github.com/tripdancer0916/pytorch-fixed-point-analysis/blob/master/model.py
        for t in range(self.n_time):output[:,t,:] = self.fc(h_all[:, t, :])

        return output, h_all , h_n

#------------------------------
# Instantiate model
model = Net(n_time, n_input, n_output, n_units_hidden, n_rnn_layers, rnn_nl).to(device)
print(model)
summary(model, verbose = 2)

#------------------------------
# Define loss function
# CrossEntropyLoss = softmax + cross entropy error
criterion =  torch.nn.MSELoss()

#------------------------------
# Define optimier
optimizer = optim.Adam(model.parameters(), lr = learning_rate)



Net(
  (rnn): RNN(1, 200, batch_first=True)
  (fc): Linear(in_features=200, out_features=1, bias=True)
)
Layer (type:depth-idx)                   Param #
Net                                      --
├─RNN: 1-1                               40,600
│    └─weight_ih_l0                      ├─200
│    └─weight_hh_l0                      ├─40,000
│    └─bias_ih_l0                        ├─200
│    └─bias_hh_l0                        └─200
├─Linear: 1-2                            201
│    └─weight                            ├─200
│    └─bias                              └─1
Total params: 40,801
Trainable params: 40,801
Non-trainable params: 0


In [10]:
#------------------------------
# Do training (make sure to get new model, criterion, optimizer)
model.train()  # change model as "training mode"

# Loop over n_epoch times
for epoch in range(n_epochs): 
    loss_sum = 0
    acc_sum = 0

    for inputs, targets in train_dataloader:        
        # CPU/GPU stuff (don't worry)
        inputs = inputs.to(device).float()
        targets = targets.to(device).float()
        
        # Initialize gradient of optimizer
        optimizer.zero_grad()

        # Forward path
        outputs, h_all, h_n = model(inputs)
        
        # Calculate loss: difference between output and label
        loss = criterion(outputs, targets)
        
        # Backward path (Backpropagation!)
        loss.backward()

        # Update weights
        optimizer.step()
        
        # Keep track of the progress of learning
        loss_sum += loss
        acc_sum += torch.sum(torch.abs(targets - outputs))

    # Display learning progress
    if  epoch % 200 == 0:
        epoch_loss = loss_sum.item() / len(train_dataloader) # cumulative loss/ batch size
        epoch_acc = acc_sum.double() / len(train_dataloader.dataset) # cummlative count of correct prediction / data size
        print(f"Epoch: {epoch+1}/{n_epochs}, Loss: {epoch_loss}, Acc: {epoch_acc}")

    # モデルの重みの保存
    #torch.save(model.state_dict(), 'model_weights.pth')

Epoch: 1/2000, Loss: 0.4736787676811218, Acc: 24.488804626464844
Epoch: 201/2000, Loss: 0.3917607069015503, Acc: 20.577940368652342
Epoch: 401/2000, Loss: 0.03259832039475441, Acc: 5.083506011962891
Epoch: 601/2000, Loss: 0.0035950392484664917, Acc: 1.9182491302490234
Epoch: 801/2000, Loss: 0.0015359095996245742, Acc: 1.266917610168457
Epoch: 1001/2000, Loss: 0.00032629817724227905, Acc: 0.5556034564971923
Epoch: 1201/2000, Loss: 0.001100083696655929, Acc: 1.0845407485961913
Epoch: 1401/2000, Loss: 0.0022626304998993874, Acc: 1.2373466491699219
Epoch: 1601/2000, Loss: 0.001049889251589775, Acc: 0.8808669090270996
Epoch: 1801/2000, Loss: 0.0003075606655329466, Acc: 0.5693301677703857
